# Resource counts

This notebook computes the resources to calculate the ground state energy of a Hamiltonian to chemical accuracy using three algorithms:

1. VQE
1. Quantum Krylov
1. Quantum phase estimation

## Setup

In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scipy

import openfermion as of
from openfermionpyscf import run_pyscf
from openfermion.chem import geometry_from_pubchem, MolecularData
import openfermion_helper


import qiskit
import qiskit.qasm3
import qiskit_ibm_runtime
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter

## Set Hamiltonian

In [2]:
geometry = [
    ("P", (-1.034220, -0.234256,0.672434)),
    ("O", (-1.004065, 0.890081, -0.334695)),
    ("O", (-0.003166, -1.329504, 0.557597)),
    ("O", (-2.065823, -0.232403, 1.765329)),
    ("H", (0.881055, 0.866924, -1.063283)),
    ("O", (1.748944, 0.417505, -1.047631)),
    ("H", (1.477276, -0.378346, -0.549750)),
]
basis = "sto-3g"
multiplicity = 1
charge = 1

molecule = MolecularData(geometry, basis, multiplicity, charge)
mol = run_pyscf(molecule, run_mp2=True, run_cisd=False, run_ccsd=False, run_fci=False)

In [3]:
mol.save()

In [4]:
hamiltonian = MolecularData(filename=molecule.filename)

In [5]:
hamiltonian = hamiltonian.get_molecular_hamiltonian()

In [6]:
hamiltonian = of.get_fermion_operator(hamiltonian)

In [7]:
len(hamiltonian.terms)

3577527

In [8]:
threshold = 0.0125
hamiltonian.compress(abs_tol=threshold)

In [9]:
len(hamiltonian.terms)

40675

In [10]:
hamiltonian_openfermion = of.jordan_wigner(hamiltonian)

In [11]:
nterms = len(hamiltonian_openfermion.terms)
nterms

18810

In [12]:
nqubits = openfermion_helper.get_num_qubits(hamiltonian_openfermion)

print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

Hamiltonian acts on 62 qubit(s) and has 18810 term(s).


In [13]:
import cirq


def preprocess_hamiltonian(
    hamiltonian: of.QubitOperator,
    drop_term_if = None,
) -> cirq.PauliSum:
    """Preprocess the Hamiltonian and convert it to a cirq.PauliSum."""
    if drop_term_if is None:
        drop_term_if = []

    new = cirq.PauliSum()

    for i, term in enumerate(hamiltonian.terms):
        print(f"Status: On term {i} = {term}", end="\r")
        add_term = True

        for drop_term in drop_term_if:
            if drop_term(term):
                add_term = False
                break

        if add_term:
            key = " ".join(pauli + str(index) for index, pauli in term)
            new += next(iter(
                of.transforms.qubit_operator_to_pauli_sum(
                    of.QubitOperator(key, hamiltonian.terms.get(term)
                )
            )))

    return new

In [14]:
hamiltonian = preprocess_hamiltonian(
    hamiltonian_openfermion, drop_term_if=[lambda term: term == ()]
)  # Drop identity.

In [15]:
# # Set parameters to make a simple molecule.
# geometry = geometry_from_pubchem("water")
# basis = "sto-3g"
# multiplicity = 1
# charge = 0

# # Make molecule.
# molecule = MolecularData(geometry, basis, multiplicity, charge)

# mol = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
# mol.save()
# hamiltonian = MolecularData(filename=molecule.filename)
# hamiltonian = hamiltonian.get_molecular_hamiltonian()
# hamiltonian = of.get_fermion_operator(hamiltonian)
# hamiltonian_openfermion = of.jordan_wigner(hamiltonian)
# hamiltonian = openfermion_helper.preprocess_hamiltonian(hamiltonian_openfermion, drop_term_if=[lambda term: term == ()])  # Drop identity.

### Show statistics

In [16]:
nterms = len(hamiltonian)
nqubits = len(hamiltonian.qubits)

print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

Hamiltonian acts on 62 qubit(s) and has 18809 term(s).


## Time evolution resources: Compute the number of CNOTs for first order Trotter

We use time evolution as a subroutine for quantum Krylov and phase estimation, so first we compute the cost of a first order Trotter step.

### (1) Crude estimate

In [17]:
# """Crude estimate."""
# num_cnots_crude: int = 0
# for term in hamiltonian:
#     num_cnots_crude += 2 ** (len(term.qubits) - 1)

# num_cnots_crude

### (2) Grouping + CNOT ladder

In [18]:
# """Estimate using grouping + diagonaliztion + exp(Z...Z) "ladder"."""
# import kcommute


# groups = kcommute.get_si_sets(hamiltonian, k=nqubits)

# num_cnots: int = 0
# for group in groups:
#     num_cnots += nqubits ** 2  # It takes O(n^2) Clifford gates to diagonalize all terms in this group [https://arxiv.org/abs/quant-ph/0406196].
#     for term in group:
#         num_cnots += 2 * len(term.qubits)  # Using 2w CNOTs in a "ladder" and one exp(Z) gate on the bottom qubit. See https://arxiv.org/abs/2408.08265v3 Fig. 3.
#     num_cnots += nqubits ** 2  # Rotating back to the Z basis (undoing the diagonal unitary).

# num_cnots

### (3) Qiskit's `PauliHedral` method

In [19]:
import convert

from qiskit.quantum_info import SparsePauliOp


def cirq_pauli_sum_to_qiskit_pauli_op(pauli_sum: cirq.PauliSum) -> SparsePauliOp:
    """Returns a qiskit.SparsePauliOp representation of the cirq.PauliSum."""
    cirq_pauli_to_str = {cirq.X: "X", cirq.Y: "Y", cirq.Z: "Z"}

    qubits = pauli_sum.qubits
    terms = []
    coeffs = []
    for i, term in enumerate(pauli_sum):
        print(f"Status: on term i = {i}", end="\r")
        string = ""
        for qubit in qubits:
            if qubit not in term:
                string += "I"
            else:
                string += cirq_pauli_to_str[term[qubit]]
        terms.append(string)
        assert np.isclose(term.coefficient.imag, 0.0, atol=1e-7)
        coeffs.append(term.coefficient.real)
    return SparsePauliOp(terms, coeffs)


H = convert.cirq_pauli_sum_to_qiskit_pauli_op(hamiltonian)

In [20]:
# Following https://qiskit-community.github.io/qiskit-algorithms/tutorials/13_trotterQRTE.html.
order: int = 1
cx_structure = "chain"  # "fountain"
trotter_step = PauliEvolutionGate(H, time=1, synthesis=LieTrotter(cx_structure=cx_structure) if order == 1 else SuzukiTrotter(order, cx_structure=cx_structure))

In [21]:
circuit = qiskit.QuantumCircuit(H.num_qubits)
circuit.append(trotter_step, range(H.num_qubits))

In [22]:
circuit = circuit.decompose(reps=2)

print(
    f"""
Depth: {circuit.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in circuit.count_ops().items()])}
"""
)
# circuit.draw(fold=-1)


Depth: 565684
Gates: CX: 506246, U2: 100064, U1: 66934, RZ: 1907, RX: 32, H: 32



### Compile

In [23]:
compiled = qiskit.transpile(
    circuit,
    optimization_level=3,
    basis_gates=["u3", "cx"]
)
print(
    f"""
Depth: {compiled.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled.count_ops().items()])}
"""
)
# compiled.draw(fold=-1)


Depth: 477541
Gates: CX: 433489, U3: 106459



### Compile to device

In [24]:
computer = qiskit_ibm_runtime.fake_provider.FakeKyiv()

compiled_kyiv = qiskit.transpile(
    compiled,
    backend=computer,
    optimization_level=3,
)
print(
    f"""
Depth: {compiled_kyiv.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled_kyiv.count_ops().items()])}
"""
)


Depth: 2911112
Gates: RZ: 2597294, SX: 1584536, ECR: 743526, X: 127621



In [ ]:
assert False

## Number of Trotter steps for chemical accuracy

See https://arxiv.org/abs/1912.08854.

In [ ]:
error = np.sum(np.abs(H.coeffs))  # Loose error bound from https://arxiv.org/abs/1912.08854.

epsilon: float = 0.001  # mHa

nsteps = round(error / epsilon)
nsteps

## VQE resources

### Gates

Read in VQE circuits.

In [ ]:
circuit_vqe = qiskit.qasm3.loads(pickle.load(open("kyiv_circuit_h2o", "rb")))

print(
    f"""
              Depth: {circuit_vqe.depth()}
         Gate count: {len(circuit_vqe)}
Nonlocal gate count: {circuit_vqe.num_nonlocal_gates()}
     Gate breakdown: {", ".join([f"{k.upper()}: {v}" for k, v in circuit_vqe.count_ops().items()])}
"""
)

## Number of shots

We compute the number of shots needed to compute one energy (cost function) $\langle \psi | H | \psi \rangle$ to accuracy $\epsilon$.

In [11]:
eval, evec = scipy.sparse.linalg.eigsh(
    of.linalg.get_sparse_operator(hamiltonian_openfermion),
    k=1,
    which="SA"
)
evec = evec.flatten()

In [ ]:
epsilons = [0.1, 0.01, 0.001]
kvals = [1, 2, nqubits // 2, nqubits]

all_shots = []
for k in kvals:
    groups = kcommute.get_si_sets(hamiltonian, k=k)
    groups_of = convert.to_groups_of(groups)
    base_shots = kcommute.compute_shots(groups_of, evec, epsilon=1)
    shots = [base_shots / epsilon ** 2 for epsilon in epsilons]
    all_shots.append(shots)
    print(all_shots)

In [ ]:
plt.rcParams.update({"font.family": "serif", "font.size": 12})


for kval, shots in zip(kvals, all_shots):
    plt.loglog(epsilons, shots, "--o", alpha=0.75, mec="black", label=f"$k = {kval}$")

plt.legend()
plt.xlabel("Accuracy $\epsilon$")
plt.ylabel("Shots $N$")
plt.title(f"$k$-commuting shot counts w.r.t. ground state\n for H2O ({nqubits} qubits, {nterms} Paulis)");